In [1]:
import pandas as pd
import numpy as np

from prophet import Prophet
from prophet.serialize import model_to_json, model_from_json

import plotly.express as px

C:\Users\User\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [15]:
df = pd.read_csv("ar41_with_speed_cleaned.csv",sep=";")
df['timestamps_UTC'] = pd.to_datetime(df['timestamps_UTC'])

In [3]:
dataset_adapted = df[["timestamps_UTC",'mapped_veh_id','RS_E_InAirTemp_PC1',"RS_T_OilTemp_PC1","RS_E_OilPress_PC1","RS_E_RPM_PC1","RS_E_WatTemp_PC1","speed","outside_temp"]]
dataset_adapted.head()

,timestamps_UTC,mapped_veh_id,RS_E_InAirTemp_PC1,RS_T_OilTemp_PC1,RS_E_OilPress_PC1,RS_E_RPM_PC1,RS_E_WatTemp_PC1,speed,outside_temp
0,2023-08-01 03:44:12,181.0,27,76.0,255.0,794.0,83.0,3,15.4
1,2023-08-01 06:36:29,143.0,33,73.0,272.0,802.0,78.0,21,16.6
2,2023-08-24 06:53:54,183.0,31,85.0,234.0,799.0,82.0,2,18.7
3,2023-08-01 13:53:38,177.0,35,78.0,220.0,794.0,77.0,54,19.4
4,2023-08-24 07:02:30,143.0,41,82.0,227.0,806.0,85.0,2,18.7


In [ ]:
selected_column = ['RS_E_InAirTemp_PC1',"RS_T_OilTemp_PC1","RS_E_OilPress_PC1","RS_E_RPM_PC1","RS_E_WatTemp_PC1"]
anomalies_df = pd.DataFrame()

for veh_id in df['mapped_veh_id'].unique():
    print(veh_id)
    veh_anomalies = df[df['mapped_veh_id'] == veh_id]
    veh_anomalies = veh_anomalies.sort_values('timestamps_UTC')
    veh_anomalies = pd.DataFrame(veh_anomalies)
    for col in selected_column:
        dataset_adapted = df[["timestamps_UTC",'mapped_veh_id','RS_E_InAirTemp_PC1',"RS_T_OilTemp_PC1","RS_E_OilPress_PC1","RS_E_RPM_PC1","RS_E_WatTemp_PC1","speed","outside_temp"]]
        dataset_adapted = dataset_adapted.rename(columns={'timestamps_UTC': 'ds',col:'y'})
        model = Prophet()
        for col_reg in dataset_adapted.columns:
            if col_reg != "ds" and col_reg != "mapped_veh_id" and col_reg != "y":
                model.add_regressor(col_reg)

        # train
        train_veh = dataset_adapted[dataset_adapted['mapped_veh_id'] == veh_id].sort_values('ds')
        train_veh.drop(columns='mapped_veh_id')
        model.fit(train_veh)

        # predict
        future = model.make_future_dataframe(periods=0) # predict on the entire dataset
        for col_reg in dataset_adapted.columns:
            if col_reg != "ds" and col_reg != "mapped_veh_id" and col_reg != "y":
                future[col_reg] = train_veh[col_reg].reset_index(drop = True)
        forecast = model.predict(future)

        forecast['y'] = train_veh['y'].reset_index(drop = True)
        forecast['error'] = forecast['y'] - forecast['yhat']
        forecast['uncertainity'] = forecast['yhat_upper'] - forecast['yhat_lower']
        threshold  = 2
        forecast['anomaly'] = forecast.apply(lambda x: 'Yes' if ( np.abs(x['error']) > threshold * x['uncertainity'] ) else 'No', axis=1)
        veh_anomalies[f"anomaly_{col}"] = np.array(forecast['anomaly'])
    anomalies_df = pd.concat([anomalies_df, veh_anomalies])
        # fig = px.scatter(forecast.reset_index(), x='ds', y='y', color='anomaly', title='SNCB outliers')
        # fig.update_xaxes(rangeslider_visible=True,
        #                 rangeselector=dict(buttons=list([
        #                     dict(count=1, label='Ja+
    #                     nuary', step='month', stepmode='backward'),
        #                     dict(count=2, label='February', step='month', stepmode='backward'),
        #                     dict(count=3, label='March', step='month', stepmode='backward'),
        #                     dict(step='all')
        #                 ])))
        # fig.write_html(f"./plot/timeseries_anomalies_veh{int(veh_id)}.html")

In [2]:
df = pd.read_csv('../anomalies.csv', sep=';')

In [ ]:
selected_column = ['RS_E_InAirTemp_PC2',"RS_T_OilTemp_PC2","RS_E_OilPress_PC2","RS_E_RPM_PC2","RS_E_WatTemp_PC2"]
anomalies_df = pd.DataFrame()

for veh_id in df['mapped_veh_id'].unique():
    print(veh_id)
    veh_anomalies = df[df['mapped_veh_id'] == veh_id]
    veh_anomalies = veh_anomalies.sort_values('timestamps_UTC')
    veh_anomalies = pd.DataFrame(veh_anomalies)
    for col in selected_column:
        dataset_adapted = df[["timestamps_UTC",'mapped_veh_id','RS_E_InAirTemp_PC2',"RS_T_OilTemp_PC2","RS_E_OilPress_PC2","RS_E_RPM_PC2","RS_E_WatTemp_PC2","speed","outside_temp"]]
        dataset_adapted = dataset_adapted.rename(columns={'timestamps_UTC': 'ds',col:'y'})
        model = Prophet()
        for col_reg in dataset_adapted.columns:
            if col_reg != "ds" and col_reg != "mapped_veh_id" and col_reg != "y":
                model.add_regressor(col_reg)

        # train
        train_veh = dataset_adapted[dataset_adapted['mapped_veh_id'] == veh_id].sort_values('ds')
        train_veh.drop(columns='mapped_veh_id')
        model.fit(train_veh)

        # predict
        future = model.make_future_dataframe(periods=0) # predict on the entire dataset
        for col_reg in dataset_adapted.columns:
            if col_reg != "ds" and col_reg != "mapped_veh_id" and col_reg != "y":
                future[col_reg] = train_veh[col_reg].reset_index(drop = True)

        forecast = model.predict(future)
        forecast['y'] = train_veh['y'].reset_index(drop = True)
        forecast['error'] = forecast['y'] - forecast['yhat']
        forecast['uncertainity'] = forecast['yhat_upper'] - forecast['yhat_lower']
        threshold  = 2
        forecast['anomaly'] = forecast.apply(lambda x: 'Yes' if ( np.abs(x['error']) > threshold * x['uncertainity'] ) else 'No', axis=1)
        veh_anomalies[f"anomaly_{col}"] = np.array(forecast['anomaly'])


    anomalies_df = pd.concat([anomalies_df, veh_anomalies])


181.0


07:09:20 - cmdstanpy - INFO - Chain [1] start processing
07:11:32 - cmdstanpy - INFO - Chain [1] done processing
07:11:59 - cmdstanpy - INFO - Chain [1] start processing
07:13:50 - cmdstanpy - INFO - Chain [1] done processing
07:14:16 - cmdstanpy - INFO - Chain [1] start processing
07:16:14 - cmdstanpy - INFO - Chain [1] done processing
07:16:42 - cmdstanpy - INFO - Chain [1] start processing
07:18:44 - cmdstanpy - INFO - Chain [1] done processing
07:19:11 - cmdstanpy - INFO - Chain [1] start processing
07:20:22 - cmdstanpy - INFO - Chain [1] done processing


143.0


07:20:49 - cmdstanpy - INFO - Chain [1] start processing
07:23:21 - cmdstanpy - INFO - Chain [1] done processing
07:23:47 - cmdstanpy - INFO - Chain [1] start processing
07:25:32 - cmdstanpy - INFO - Chain [1] done processing
07:25:58 - cmdstanpy - INFO - Chain [1] start processing
07:27:33 - cmdstanpy - INFO - Chain [1] done processing
07:27:59 - cmdstanpy - INFO - Chain [1] start processing
07:29:47 - cmdstanpy - INFO - Chain [1] done processing
07:30:13 - cmdstanpy - INFO - Chain [1] start processing
07:32:00 - cmdstanpy - INFO - Chain [1] done processing


183.0


...

09:41:41 - cmdstanpy - INFO - Chain [1] start processing
09:44:17 - cmdstanpy - INFO - Chain [1] done processing
09:44:43 - cmdstanpy - INFO - Chain [1] start processing
09:48:26 - cmdstanpy - INFO - Chain [1] done processing
09:48:53 - cmdstanpy - INFO - Chain [1] start processing
09:50:56 - cmdstanpy - INFO - Chain [1] done processing
09:51:23 - cmdstanpy - INFO - Chain [1] start processing
09:52:36 - cmdstanpy - INFO - Chain [1] done processing
09:53:03 - cmdstanpy - INFO - Chain [1] start processing
09:56:58 - cmdstanpy - INFO - Chain [1] done processing


161.0


09:57:22 - cmdstanpy - INFO - Chain [1] start processing
09:58:08 - cmdstanpy - INFO - Chain [1] done processing
09:58:19 - cmdstanpy - INFO - Chain [1] start processing
09:58:42 - cmdstanpy - INFO - Chain [1] done processing
09:58:54 - cmdstanpy - INFO - Chain [1] start processing
10:00:17 - cmdstanpy - INFO - Chain [1] done processing
10:00:28 - cmdstanpy - INFO - Chain [1] start processing
10:01:36 - cmdstanpy - INFO - Chain [1] done processing
10:01:47 - cmdstanpy - INFO - Chain [1] start processing
10:02:06 - cmdstanpy - INFO - Chain [1] done processing


142.0


10:02:22 - cmdstanpy - INFO - Chain [1] start processing
10:04:16 - cmdstanpy - INFO - Chain [1] done processing
10:04:46 - cmdstanpy - INFO - Chain [1] start processing
10:07:31 - cmdstanpy - INFO - Chain [1] done processing
10:08:02 - cmdstanpy - INFO - Chain [1] start processing


In [ ]:
anomalies_df

In [ ]:
anomalies_df = anomalies_df.drop(columns='Unnamed: 0')

In [ ]:
anomalies_df.to_csv('anomalies.csv',sep=';')

In [2]:
anomalies_df = pd.read_csv('anomalies.csv', sep=';')

In [5]:
len(anomalies_df)

16867540

In [11]:
anomalies_df = anomalies_df.drop(columns=['Unnamed: 0','Unnamed: 0.1'])

In [13]:
anomalies_df[anomalies_df['mapped_veh_id']==182]

,mapped_veh_id,timestamps_UTC,lat,lon,RS_E_InAirTemp_PC1,RS_E_InAirTemp_PC2,RS_E_OilPress_PC1,RS_E_OilPress_PC2,RS_E_RPM_PC1,RS_E_RPM_PC2,...,station_name,province_name,outside_temp,weather,speed,anomaly_RS_E_InAirTemp_PC1,anomaly_RS_T_OilTemp_PC1,anomaly_RS_E_OilPress_PC1,anomaly_RS_E_RPM_PC1,anomaly_RS_E_WatTemp_PC1
2662100,182.0,2023-01-23 08:24:04,51.035656,3.710717,16,9,207.0,224.0,806.0,799.0,...,Ghent Sint Pieters,Oost-Vlaanderen,1.7,Broken clouds,0,No,No,No,No,No
2662101,182.0,2023-01-23 08:24:56,51.035410,3.711797,16,8,210.0,224.0,797.0,799.0,...,Ghent Sint Pieters,Oost-Vlaanderen,1.7,Broken clouds,5,No,No,No,No,No
2662102,182.0,2023-01-23 08:25:57,51.035407,3.711814,17,8,210.0,224.0,798.0,798.0,...,Ghent Sint Pieters,Oost-Vlaanderen,1.7,Broken clouds,0,No,No,No,No,No
2662103,182.0,2023-01-23 08:30:00,51.035400,3.711835,17,7,217.0,227.0,802.0,799.0,...,Ghent Sint Pieters,Oost-Vlaanderen,1.7,Broken clouds,0,No,No,No,No,No
2662104,182.0,2023-01-23 08:31:00,51.035399,3.711808,16,7,220.0,231.0,801.0,802.0,...,Ghent Sint Pieters,Oost-Vlaanderen,1.9,Broken clouds,0,No,No,No,No,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2792964,182.0,2023-09-13 16:24:15,50.768640,3.870845,31,36,179.0,179.0,803.0,801.0,...,Geraardsbergen,Oost-Vlaanderen,19.7,Broken clouds,21,No,No,No,No,No
2792965,182.0,2023-09-13 16:25:16,50.771994,3.872430,33,35,179.0,175.0,799.0,800.0,...,Geraardsbergen,Oost-Vlaanderen,19.7,Broken clouds,22,No,No,No,No,No
2792966,182.0,2023-09-13 16:26:16,50.774384,3.872867,33,35,179.0,179.0,799.0,799.0,...,NaN,Oost-Vlaanderen,19.7,Broken clouds,16,No,No,No,No,No
2792967,182.0,2023-09-13 16:26:37,50.774389,3.872878,35,35,17.0,51.0,142.0,142.0,...,NaN,Oost-Vlaanderen,19.7,Broken clouds,0,No,No,No,No,No


In [10]:
anomalies_df[pd.isna(anomalies_df['anomaly_RS_E_InAirTemp_PC1'])]

,Unnamed: 0.1,Unnamed: 0,mapped_veh_id,timestamps_UTC,lat,lon,RS_E_InAirTemp_PC1,RS_E_InAirTemp_PC2,RS_E_OilPress_PC1,RS_E_OilPress_PC2,...,station_name,province_name,outside_temp,weather,speed,anomaly_RS_E_InAirTemp_PC1,anomaly_RS_T_OilTemp_PC1,anomaly_RS_E_OilPress_PC1,anomaly_RS_E_RPM_PC1,anomaly_RS_E_WatTemp_PC1


In [26]:
(len(forecast[forecast['anomaly'] == 'Yes'])/len(forecast))*100

0.35511925899955893

In [27]:
len(forecast)

88421

In [6]:
# Save model
# with open('serialized_model.json', 'w') as fout:
#     fout.write(model_to_json(my_model))
# Load model
# with open('serialized_model.json', 'r') as fin:
#    my_model = model_from_json(fin.read())

In [31]:
test_set = df[~(df['mapped_veh_id'].isin(vehicles_id_train))]
test_set = test_set[["timestamps_UTC","mapped_veh_id",'RS_E_InAirTemp_PC1',"RS_T_OilTemp_PC1","RS_E_OilPress_PC1","RS_E_RPM_PC1","RS_E_WatTemp_PC1","speed","outside_temp"]]
# test_set['timestamps_UTC'] = test_set['timestamps_UTC'].apply(lambda x : datetime.strptime(x ,'%Y-%m-%d %H:%M:%S'))
test_set = test_set.rename(columns={'timestamps_UTC': 'ds'})

In [32]:
test_set = pd.DataFrame(test_set)
all_predictions = {}

for veh_id, veh_df in test_set.groupby('mapped_veh_id'):
    print(veh_id)
    veh_df = veh_df.drop(columns='mapped_veh_id')
    predictions_batch = my_model.predict(veh_df)
    all_predictions[veh_id] = predictions_batch

102.0
104.0
105.0
107.0
108.0
109.0
110.0
112.0
114.0
119.0
121.0
122.0
123.0
124.0
126.0
127.0
128.0
129.0
133.0
134.0
137.0
138.0
139.0
142.0
143.0
145.0
148.0
150.0
152.0
160.0
162.0
164.0
165.0
167.0
169.0
173.0
174.0
176.0
178.0
182.0
184.0
187.0
188.0
189.0
191.0
194.0


In [48]:
forecast = all_predictions[104]
forecast['y'] = train_set['RS_E_InAirTemp_PC1'].reset_index(drop = True)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper','y']].head()
forecast['error'] = forecast['y'] - forecast['yhat']
forecast['uncertainity'] = forecast['yhat_upper'] - forecast['yhat_lower']
threshold  = 2

# forecast[forecast['error'].abs() > threshold * forecast['uncertainity']].head()
#---------------------------Plot------------------------------------------
forecast['anomaly'] = forecast.apply(lambda x: 'Yes' if ( np.abs(x['error']) > threshold * x['uncertainity'] ) else 'No', axis=1)

# forecast['anomaly'] = forecast.apply(lambda x: 'yes' if ( x['y'] > x['yhat_upper'] or x['y'] < x['yhat_lower'] ) else 'No', axis=1)

fig = px.scatter(forecast.reset_index(), x='ds', y='y',
                color='anomaly', title='SNCB outliers')
fig.update_xaxes(rangeslider_visible=True,
                rangeselector=dict(buttons=list([
                    dict(count=1, label='January', step='month', stepmode='backward'),
                    dict(count=2, label='February', step='month', stepmode='backward'),
                    dict(count=3, label='March', step='month', stepmode='backward'),
                    dict(step='all')
                ])))
fig.write_html("./plot/timeseries_anomalies.html")

In [49]:
forecast[forecast['anomaly'] == 'Yes']

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,RS_E_OilPress_PC1,RS_E_OilPress_PC1_lower,RS_E_OilPress_PC1_upper,RS_E_RPM_PC1,...,weekly_lower,weekly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat,y,error,uncertainity,anomaly
884,2023-01-23 23:04:57,28.960456,5.722635,25.340389,28.960456,28.960456,-4.944087,-4.944087,-4.944087,-1.127617,...,0.192061,0.192061,0.0,0.0,0.0,15.745096,60,44.254904,19.617754,Yes
892,2023-01-23 23:12:59,28.959727,4.702397,24.553586,28.959727,28.959727,-4.791004,-4.791004,-4.791004,-1.102971,...,0.193105,0.193105,0.0,0.0,0.0,14.940065,55,40.059935,19.851190,Yes
938,2023-01-23 23:59:51,28.955470,1.504736,20.987397,28.955470,28.955470,-4.676192,-4.676192,-4.676192,-1.135832,...,0.198571,0.198571,0.0,0.0,0.0,11.594793,64,52.405207,19.482661,Yes
979,2023-01-24 00:41:00,28.951733,-0.568479,18.630520,28.951733,28.951733,-4.791004,-4.791004,-4.791004,-1.111186,...,0.202491,0.202491,0.0,0.0,0.0,9.163511,57,47.836489,19.198998,Yes
1017,2023-01-24 01:19:58,28.948194,-2.148661,17.062535,28.948194,28.948194,-4.676192,-4.676192,-4.676192,-1.102971,...,0.205450,0.205450,0.0,0.0,0.0,7.713132,47,39.286868,19.211196,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102548,2023-08-28 01:03:48,32.017575,-2.892779,17.074256,32.017575,32.017575,-10.340264,-10.340264,-10.340264,-1.102971,...,-0.242015,-0.242015,0.0,0.0,0.0,6.496921,47,40.503079,19.967035,Yes
102576,2023-08-28 02:31:59,32.018666,3.618257,23.691638,32.018666,32.018666,-5.747773,-5.747773,-5.747773,-1.201553,...,-0.212328,-0.212328,0.0,0.0,0.0,13.531157,56,42.468843,20.073381,Yes
104012,2023-09-01 03:25:28,32.090600,2.057064,21.856769,32.090600,32.090600,-9.421766,-9.421766,-9.421766,-1.152262,...,0.032124,0.032124,0.0,0.0,0.0,12.099606,52,39.900394,19.799705,Yes
105230,2023-09-05 17:54:32,32.172626,34.014807,54.275380,32.172626,32.172626,2.059461,2.059461,2.059461,-1.135832,...,0.102504,0.102504,0.0,0.0,0.0,43.926930,3,-40.926930,20.260572,Yes


In [14]:
forecast[forecast['anomaly'] == 'f']

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,RS_E_OilPress_PC1,RS_E_OilPress_PC1_lower,RS_E_OilPress_PC1_upper,RS_E_RPM_PC1,...,weekly_lower,weekly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat,y,error,uncertainity,anomaly
16,2023-01-23 08:46:44,28.142466,13.691419,32.320539,28.142466,28.142466,1.767308,1.767308,1.767308,-1.173607,...,-0.050163,-0.050163,0.0,0.0,0.0,23.067562,56,32.932438,18.629120,yes
18,2023-01-23 08:48:45,28.142582,14.225319,33.511483,28.142582,28.142582,1.882914,1.882914,1.882914,-1.192314,...,-0.049473,-0.049473,0.0,0.0,0.0,23.541509,55,31.458491,19.286163,yes
72,2023-01-23 09:39:43,28.145499,16.425543,35.865254,28.145499,28.145499,2.538016,2.538016,2.538016,-0.827537,...,-0.032072,-0.032072,0.0,0.0,0.0,25.967296,65,39.032704,19.439711,yes
75,2023-01-23 09:42:47,28.145674,15.732895,34.792208,28.145674,28.145674,-3.434969,-3.434969,-3.434969,4.475750,...,-0.031028,-0.031028,0.0,0.0,0.0,24.913277,60,35.086723,19.059313,yes
113,2023-01-23 10:19:48,28.147793,15.013557,33.119388,28.147793,28.147793,1.767308,1.767308,1.767308,-1.145548,...,-0.018452,-0.018452,0.0,0.0,0.0,23.885625,52,28.114375,18.105832,yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107358,2023-09-07 20:40:42,32.238775,30.424705,49.898365,32.238775,32.238775,1.998521,1.998521,1.998521,-1.145548,...,0.007355,0.007355,0.0,0.0,0.0,40.347424,11,-29.347424,19.473660,yes
107543,2023-09-08 13:59:25,32.250641,18.971402,37.790002,32.250641,32.250641,-5.284668,-5.284668,-5.284668,-1.164254,...,0.097140,0.097140,0.0,0.0,0.0,28.847723,63,34.152277,18.818600,yes
107627,2023-09-12 05:52:14,32.310876,2.156333,20.671100,32.310876,32.310876,-11.084244,-11.084244,-11.084244,-1.178284,...,0.257645,0.257645,0.0,0.0,0.0,11.239890,43,31.760110,18.514767,yes
107745,2023-09-12 15:40:50,32.317600,11.358505,30.372730,32.317600,32.317600,-8.213358,-8.213358,-8.213358,-1.173607,...,0.263662,0.263662,0.0,0.0,0.0,20.491535,57,36.508465,19.014226,yes
